# 通过聚类识别登陆养卡群体

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MultipleLocator,FormatStrFormatter
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 连接hive 数据库

In [ ]:
from pyhive import hive
cursor = hive.connect(host='192.168.1.106',post=10000,
                username='root',database='zjyd3').cursor()
cursor.execute('select * from temp_wqf_log_info_04')
a = cursor.fetchall()
alldata = pd.DataFrame(a,columns=['phonenumber','sl','ips_pro','imeis_pro','timecv','fail_pro'])
del a 

## 数据标准化

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_data = alldata[['sl','ips_pro','imeis_pro','timecv']]
train_data.fillna(0,inplace=True)
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)

## 模型选择优化

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabaz_score
n_class = []
ch_values = []
for i in range(10):
    n_cluster = i + 2
    print(n_class)
    clf = KMeans(n_clusters=n_cluster,random_state=0)
    label = clf.fit_predict(train_data)
    ch_values.append(calinski_harabaz_score(train_data,label))
    n_class.append(n_cluster)
plt.plot(n_class,ch_values,label='First line',linewidth=3,
         color='r',marker='o',markerfacecolor='blue',
         markersize=12)
ax = plt.subplot(111)
x_major_Locator = MultipleLocator(1)
x_major_Formatter = FormatStrFormatter('%5.0f')
ax.xaxis.set_major_locator(x_major_Locator)
ax.xaxis.set_major_formatter(x_major_Formatter)
plt.xlabel('n_class')
plt.ylabel('ch values')